In [39]:
%run uniswap_trader.ipynb

In [40]:
from web3 import Web3
from web3.gas_strategies.rpc import rpc_gas_price_strategy
from uniswap import Uniswap
import time
from dotenv import load_dotenv
import os
import json
import requests
import numpy as np

In [41]:
class RetikTrader(TokenTrader):
    def __init__(self, rpc_url, wallet_address, private_key, token_address, usdt_address,
                 eth_address, price_thresholds, sale_percentages, min_sell_amount_tokens, 
                 etherscan_api_key, presale_contract_address, presale_id, lot_size_tokens):
        
        # Llamada correcta a super con todos los argumentos necesarios
        super().__init__(rpc_url, wallet_address, private_key, token_address, 
                         usdt_address, eth_address, price_thresholds, sale_percentages, 
                         min_sell_amount_tokens, etherscan_api_key)

        self.presale_contract_address = presale_contract_address
        self.presale_id = presale_id
        self.presale_contract = self.web3.eth.contract(address=Web3.to_checksum_address(self.presale_contract_address), abi=self.get_contract_abi())
        self.lot_size = int(lot_size_tokens * 10**self.token_decimals)
        
    def get_contract_abi(self):
        try:
            url = f"https://api.etherscan.io/api?module=contract&action=getabi&address={self.presale_contract_address}&apikey={self.etherscan_api_key}"
            response = requests.get(url)
            response_json = response.json()
            # Comprueba si la solicitud fue exitosa y si hay un ABI disponible
            if response_json['status'] == '1' and response_json['message'] == 'OK':
                abi = json.loads(response_json['result'])
                return abi
        except Exception as e:
            print(f"Ha ocurrido un error al obtener el ABI: {e}")
            return None
    def claimable_tokens(self):
        # Obtener datos desde blockchain
        user_data = self.contract.functions.userClaimData(self.wallet_address, self.presale_id).call()
        vesting_data = self.contract.functions.vesting(self.presale_id).call()
        # Estructurar los datos que se necesitan
        current_time = self.web3.eth.get_block('latest').timestamp
        start_time = vesting_data[0]
        initial_percent = vesting_data[1]
        vesting_time = vesting_data[2]
        vesting_percent = vesting_data[3]
        total_cycles = vesting_data[4]
        total_tokens = user_data[2]
        # Condiciones para continuar con el cálculo
        if not (vesting_time > 0):
            return 0
        if current_time < start_time:
            return 0
        # Cálculo de los tokens reclamables según tiempo
        elapsed_time = current_time - start_time
        cycles_completed = min(elapsed_time // vesting_time, total_cycles)
        initially_claimable = total_tokens * initial_percent / 1000
        additionally_claimable = (total_tokens * vesting_percent / 1000) * cycles_completed
        claimable_now = initially_claimable + additionally_claimable - user_data[4]
        return max(0, claimable_now)
    
    def claim_tokens(self):
        claim_txn = self.contract.functions.claimAmount(self.presale_id).buildTransaction({
            'from': self.wallet_address,
            'nonce': self.web3.eth.get_transaction_count(self.wallet_address)
        })
        signed_txn = self.web3.eth.account.sign_transaction(claim_txn, private_key=self.private_key)
        tx_hash = self.web3.eth.send_raw_transaction(signed_txn.rawTransaction)
        receipt = self.web3.eth.wait_for_transaction_receipt(tx_hash)
        print(f"Tokens reclamados con éxito, transacción: {receipt.transactionHash.hex()}")
    
    def tokens_in_staking(self):
        # Aquí se debe llamar al contrato de staking para obtener los tokens que tienes en stake
        # Suponiendo que el contrato de staking tiene un método getStakedTokens que devuelve la cantidad de tokens
        staked_tokens = self.staking_contract.functions.getStakedTokens(self.wallet_address).call()
        return staked_tokens

    def stake_tokens(self, amount):
        # Aquí se llama al contrato de staking para stakear los tokens
        stake_txn = self.staking_contract.functions.stake(amount).buildTransaction({
            'from': self.wallet_address,
            'nonce': self.web3.eth.get_transaction_count(self.wallet_address)
        })
        signed_txn = self.web3.eth.account.sign_transaction(stake_txn, private_key=self.private_key)
        self.web3.eth.send_raw_transaction(signed_txn.rawTransaction)
        print(f"Staked {amount} tokens")

    def unstake_tokens(self, amount):
        # Aquí se llama al contrato de staking para unstakear los tokens
        unstake_txn = self.staking_contract.functions.unstake(amount).buildTransaction({
            'from': self.wallet_address,
            'nonce': self.web3.eth.get_transaction_count(self.wallet_address)
        })
        signed_txn = self.web3.eth.account.sign_transaction(unstake_txn, private_key=self.private_key)
        self.web3.eth.send_raw_transaction(signed_txn.rawTransaction)
        print(f"Unstaked {amount} tokens")
    
    def manage_sales(self, balance, price_in_usdt):
        total_tokens = balance + self.claimable_tokens() + self.tokens_in_staking()  # Total tokens incluyendo staking
        total_to_sell, lots_to_sell = self.calculate_sale_amount(price_in_usdt, total_tokens)

        # Si hay suficientes tokens disponibles para vender directamente o después de unstaking
        if balance + self.claimable_tokens() >= total_to_sell:
            if self.claimable_tokens() > 0:
                self.claim_tokens()
            self.sell_tokens(total_to_sell, price_in_usdt, lots_to_sell)
        elif balance + self.claimable_tokens() + self.tokens_in_staking() >= total_to_sell:
            needed_from_staking = total_to_sell - (balance + self.claimable_tokens())
            self.unstake_tokens(needed_from_staking)
            if self.claimable_tokens() > 0:
                self.claim_tokens()
            self.sell_tokens(total_to_sell, price_in_usdt, lots_to_sell)
        else:
            # Considerar stakear los tokens disponibles si no son suficientes para vender
            if balance + self.claimable_tokens() > 0:
                if self.claimable_tokens() > 0:
                    self.claim_tokens()
                self.stake_tokens(balance + self.claimable_tokens())
                print(f"Staking {(balance + self.claimable_tokens()) / 10**self.token_decimals} tokens")


In [42]:
if __name__ == "__main__":
    # Cargar variables de entorno
    load_dotenv()
    
    # Configuración
    rpc_url = "https://rpc.ankr.com/eth" 
    wallet_address = os.getenv("WALLET_ADDRESS")
    private_key = os.getenv("PRIVATE_KEY")
    etherscan_api_key = os.getenv("ETHERSCAN_API_KEY")
    eth_address = "0x0000000000000000000000000000000000000000"
    usdt_address = "0xdAC17F958D2ee523a2206206994597C13D831ec7"
    token_address = "0x26EbB8213fb8D66156F1Af8908d43f7e3e367C1d"
    token_contract = "0x602C90D796D746b97a36f075d9f3b2892B9B07c2"
    presale_id = 2
    price_thresholds = np.linspace(0.19, 1.99, 20)
    sale_percentages = np.linspace(0.05, 0.1, len(price_thresholds))
    lot_size_tokens = 2547
    min_sell_amount_tokens = 1000

    trader = RetikTrader(rpc_url, wallet_address, private_key, token_address, usdt_address, eth_address, price_thresholds,
                         sale_percentages, min_sell_amount_tokens, etherscan_api_key, token_contract, presale_id, lot_size_tokens)
    trader.trade()

Balance actual: 0.0 tokens
Ha ocurrido un error al chequear el precio: ('execution reverted', 'no data')
Precio actual: None USDT
Balance actual: 0.0 tokens
Ha ocurrido un error al chequear el precio: ('execution reverted', 'no data')
Precio actual: None USDT
Balance actual: 0.0 tokens
Ha ocurrido un error al chequear el precio: ('execution reverted', 'no data')
Precio actual: None USDT


KeyboardInterrupt: 